In [20]:
import duckdb

### Question 1
We need to find player with no of gold medals won by them only for players who won only gold medals.

In [21]:
query = """
CREATE TABLE events (
ID int,
event varchar(255),
YEAR INt,
GOLD varchar(255),
SILVER varchar(255),
BRONZE varchar(255)
);

delete from events;

INSERT INTO events VALUES (1,'100m',2016, 'Amthhew Mcgarray','donald','barbara');
INSERT INTO events VALUES (2,'200m',2016, 'Nichole','Alvaro Eaton','janet Smith');
INSERT INTO events VALUES (3,'500m',2016, 'Charles','Nichole','Susana');
INSERT INTO events VALUES (4,'100m',2016, 'Ronald','maria','paula');
INSERT INTO events VALUES (5,'200m',2016, 'Alfred','carol','Steven');
INSERT INTO events VALUES (6,'500m',2016, 'Nichole','Alfred','Brandon');
INSERT INTO events VALUES (7,'100m',2016, 'Charles','Dennis','Susana');
INSERT INTO events VALUES (8,'200m',2016, 'Thomas','Dawn','catherine');
INSERT INTO events VALUES (9,'500m',2016, 'Thomas','Dennis','paula');
INSERT INTO events VALUES (10,'100m',2016, 'Charles','Dennis','Susana');
INSERT INTO events VALUES (11,'200m',2016, 'jessica','Donald','Stefeney');
INSERT INTO events VALUES (12,'500m',2016,'Thomas','Steven','Catherine');"""
duckdb.execute(query)
query = """SELECT * FROM events"""
duckdb.sql(query).show()

CatalogException: Catalog Error: Table with name "events" already exists!

#### Solution

In [ ]:
query = """
SELECT GOLD,COUNT(*) FROM events 
WHERE GOLD NOT IN (SELECT BRONZE FROM events UNION ALL SELECT SILVER FROM events)
GROUP BY GOLD
"""

duckdb.sql(query).show()

┌──────────────────┬──────────────┐
│       GOLD       │ count_star() │
│     varchar      │    int64     │
├──────────────────┼──────────────┤
│ Amthhew Mcgarray │            1 │
│ Charles          │            3 │
│ Ronald           │            1 │
│ Thomas           │            3 │
│ jessica          │            1 │
└──────────────────┴──────────────┘



#### Alternative Solution

In [ ]:
query = """
WITH only_gold AS (
    SELECT GOLD
    FROM events
    EXCEPT
    SELECT SILVER
    FROM events
    EXCEPT
    SELECT BRONZE
    FROM events
)
SELECT events.GOLD,
    COUNT(*)
FROM events
    INNER JOIN only_gold ON events.GOLD = only_gold.GOLD
GROUP BY events.GOLD
"""
duckdb.sql(query).show()

┌──────────────────┬──────────────┐
│       GOLD       │ count_star() │
│     varchar      │    int64     │
├──────────────────┼──────────────┤
│ Amthhew Mcgarray │            1 │
│ Charles          │            3 │
│ Ronald           │            1 │
│ Thomas           │            3 │
│ jessica          │            1 │
└──────────────────┴──────────────┘

